In [1]:
# Author: Kilian Holzapfel <kilian.holzapfel@tum.de>

import sdaq
import h5py
import time
import numpy as np
import os
import logging

# Setup the logger
sdaq is intended to work in the background with threads and also over long periods. Therefore logging is an important part (espacially to files) for debugging.

In [2]:
logging_file = 'sdaq.log'
sdaq.setup_logging(
    # enables the file logger to the specified file
    file_name=logging_file,
    # sets the logging level for the file logger and also for the console logger if to_console=True
    log_level=logging.DEBUG,  # sets the logging level for the file logger and also for the console
    # enables the logging to the console with the specified level
    to_console=logging.INFO,
    )

<RootLogger root (DEBUG)>

# Define getter functions
Returns random numbers here, but can be any function that returns a measurement when it gets called. SDAQ executes the functions automatically with the specified frequency in the background.

In [3]:
def getter():
    """A dummy-getter function which returns one value."""
    return [np.random.random()]


def getter_2():
    """A dummy-getter function which returns one value and a 2x2 array."""
    return [np.random.random(), np.random.random((2, 2))]

getter(), getter_2()
sdaq.DAQJobExtern

sdaq.daq_job.DAQJobExtern

# Initialize the single Jobs and the Daemon
- A single **DAQJob** is basically a single time-series of data. `read_period` defines the frequency of the execution of the `getter`-function. But it is also possible to disable the scheduler and trigger a `getter`-execution somewhere else in the code. It is also possible that the `getter` is another program with `sdaq.DAQJobExtern`. The program gets executed and the package reads the data from the STDOUT stream.
- The **DAQDaemon** wrties the data to the file and if specified also handles the rollover to a new file. It collects the data from its DAQJobs (and empties the DAQJobs buffer) and writes it to the file.

In [4]:
# set up Jobs
job_list = [sdaq.DAQJob('job_1', 'table_0', getter),
            sdaq.DAQJob('job_2', ['1d_array', 'nd_array'], getter_2, shape=[(0,), (0, 2, 2)],read_period=.2)
            ]
# set up Daemon
daq_daemon = sdaq.DAQDaemon(job_list)

# start the Daemon, wait, and stop it
daq_daemon.start(attrs={'measurement': 'test', 'run_id': 12})  # attrs are added as root attributes of the hdf5 file
time.sleep(3)
daq_daemon.stop()

2024-01-26 11:41:39,573;   INFO;ThreadScheduler_DAQDaemon-: Stop scheduler; DAQDaemon-
2024-01-26 11:41:39,574;   INFO;ThreadScheduler_DAQDaemon-: Start scheduler; DAQDaemon-
2024-01-26 11:41:42,576;   INFO;          DAQDaemon-: Stop DAQDaemon , active: True
2024-01-26 11:41:42,577;   INFO;ThreadScheduler_DAQDaemon-: Stop scheduler; DAQDaemon-
2024-01-26 11:41:42,599;   INFO;          DAQDaemon-: rename file from /Users/kilian/PycharmProjects/sdaq/examples/DAQ_20240126T104139.572Z_20240126T235959.999Z.hdf5 to /Users/kilian/PycharmProjects/sdaq/examples/DAQ_20240126T104139.572Z_20240126T104142.577Z.hdf5


'Stopping the logger daemon - file renamed from /Users/kilian/PycharmProjects/sdaq/examples/DAQ_20240126T104139.572Z_20240126T235959.999Z.hdf5 to /Users/kilian/PycharmProjects/sdaq/examples/DAQ_20240126T104139.572Z_20240126T104142.577Z.hdf5'

# Checkout the logger file

In [5]:
with open(logging_file, 'r') as f:
    for i in f.readlines():
        print(i.strip())

DEBUG:ThreadScheduler_DAQDaemon-:__del__
DEBUG:ThreadScheduler_DAQDaemon-:Clear DAQDaemon- scheduler.
INFO:ThreadScheduler_DAQDaemon-:Stop scheduler; DAQDaemon-
INFO:ThreadScheduler_DAQDaemon-:Stop scheduler; DAQDaemon-
DEBUG:ThreadScheduler_DAQDaemon-:__del__
DEBUG:ThreadScheduler_DAQDaemon-:Clear DAQDaemon- scheduler.
INFO:ThreadScheduler_DAQDaemon-:Stop scheduler; DAQDaemon-
INFO:ThreadScheduler_DAQDaemon-:Stop scheduler; DAQDaemon-
DEBUG:ThreadScheduler_DAQDaemon-:__del__
DEBUG:ThreadScheduler_DAQDaemon-:Clear DAQDaemon- scheduler.
INFO:ThreadScheduler_DAQDaemon-:Stop scheduler; DAQDaemon-
INFO:ThreadScheduler_DAQDaemon-:Stop scheduler; DAQDaemon-
2024-01-26 11:11:42,490;INFO;Stop scheduler; DAQDaemon-
2024-01-26 11:11:42,492;INFO;Start scheduler; DAQDaemon-
2024-01-26 11:11:45,498;INFO;Stop DAQDaemon , active: True
2024-01-26 11:11:45,501;INFO;Stop scheduler; DAQDaemon-
2024-01-26 11:11:45,519;INFO;rename file from /Users/kilian/PycharmProjects/sdaq/examples/DAQ_20240126T101142.48

# Checkout what is writen in the hdf5 file

In [6]:
# check the file
with h5py.File(daq_daemon.file_name, mode='r') as f:
    print(f'{f.filename}')
    for i in f:
        print(f'---> {i}')
        for j in f.get(i):
            data = f.get(f'{i}/{j}')[:]
            print(f'{i+"/"+j:12} \t- shape: {data.shape}')
            if j == 'time':
                t_data = (data * 1e9).astype("datetime64[ns]")
                print(f'period: {t_data[0]} - {t_data[-1]}')
        print()

/Users/kilian/PycharmProjects/sdaq/examples/DAQ_20240126T104139.572Z_20240126T104142.577Z.hdf5
---> job_1
job_1/table_0 	- shape: (4,)
job_1/time   	- shape: (4,)
period: 2024-01-26T09:41:39.573252864 - 2024-01-26T09:41:42.575230976

---> job_2
job_2/1d_array 	- shape: (15,)
job_2/nd_array 	- shape: (15, 2, 2)
job_2/time   	- shape: (15,)
period: 2024-01-26T09:41:39.573492992 - 2024-01-26T09:41:42.427791104



In [7]:
# delete the file
os.remove(daq_daemon.file_name)